In [7]:
import pandas as pd
import datetime as dt
import numpy as np
import ast

IMPORT ET TRI TITLE PRINCIPALS

In [113]:
df_title_principals = pd.read_csv('BD/title.principals.tsv.gz', compression = 'gzip', sep = '\t')

In [114]:
df_title_principals = df_title_principals[(df_title_principals['category'] == 'actor') | (df_title_principals['category'] == 'actress') | (df_title_principals['category'] == 'self') | (df_title_principals['category'] == 'director')]

IMPORT DF FILMS ET TRI COLONNES

In [ ]:
df_films = pd.read_csv('BD/P2_G5_films.csv.gz', compression = 'gzip')

In [ ]:
liste_films = df_films['film_id_out_KNN'].tolist()

TRI

In [ ]:
df_title_principals['a_garder'] = df_title_principals['tconst'].apply(lambda x : x in liste_films)

In [ ]:
df_title_principals = df_title_principals[df_title_principals['a_garder'] == True]

In [ ]:
df_title_principals.to_csv('P2_G5_base_name.csv.gz', index=False, compression='gzip')

IMPORT ET EXPLODE NAME BASICS

In [8]:
df_name_basics = pd.read_csv('BD/name.basics.tsv.gz', compression = 'gzip', sep = '\t')

In [9]:
df_name_basics['knownForTitles'] = df_name_basics['knownForTitles'].apply(lambda x : x.split(','))
df_name_basics_exp = df_name_basics.explode('knownForTitles')

In [10]:
df_name_basics_exp = df_name_basics.explode('knownForTitles')

In [11]:
df_name_basics = 0

IMPORT ET EXPLODE TITLE CREW

In [17]:
df_title_crew = pd.read_csv('BD/title.crew.tsv.gz', compression = 'gzip', sep = '\t')

In [20]:
df_title_crew['directors'] = df_title_crew['directors'].apply(lambda x : x.split(','))
df_title_crew_exp = df_title_crew.explode('directors')

In [21]:
df_title_crew_exp = df_title_crew.explode('directors')

In [28]:
df_title_crew = 0

In [49]:
df_title_crew_exp = df_title_crew_exp.rename({'directors' : 'nconst'}, axis = 1)

In [30]:
# df_title_crew_exp['directors'] = df_title_crew_exp['directors'].apply(lambda x : ''.join(x))

In [42]:
df1 = pd.merge(df_films,
               df_title_principals,
               how = 'left',
               left_on = 'film_id_out_KNN',
               right_on = 'tconst')

In [44]:
df_films, df_title_principals = 0, 0

RECHERCHE DES REALISATEURS

In [46]:
df_dir = df1[df1['category'] == 'director']
df_autre = df1[~(df1['category'] == 'director')]

In [53]:
df_dir_merge = pd.merge(df_dir,
                        df_title_crew_exp,
                        how = 'left',
                        on = ['tconst', 'nconst'])

In [62]:
df2 = pd.concat([df_dir_merge, df_autre])

In [66]:
df1, df_dir_merge, df_dir, df_title_crew_exp, df_autre = 0, 0, 0, 0, 0

In [70]:
df2 = df2.drop('writers', axis = 1)

AJOUT DES NAMES BASICS

In [81]:
df_final = pd.merge(df2,
                    df_name_basics_exp,
                    how = 'left',
                    left_on = ['film_id_out_KNN', 'nconst'],
                    right_on = ['knownForTitles', 'nconst'])

In [82]:
len(df_final)

545434

In [83]:
len(df2)

545434

In [96]:
essai1 = df_final[df_final['primaryName'].str.contains('James Cameron') == True]

In [97]:
essai1

,film_id_out_KNN,tconst,ordering,nconst,category,job,characters,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
2542,tt0499549,tt0499549,11.0,nm0000116,director,\N,\N,James Cameron,1954,\N,"writer,producer,director",tt0499549
11970,tt1630029,tt1630029,11.0,nm0000116,director,\N,\N,James Cameron,1954,\N,"writer,producer,director",tt1630029
22697,tt0090605,tt0090605,11.0,nm0000116,director,director,\N,James Cameron,1954,\N,"writer,producer,director",tt0090605
33084,tt0103064,tt0103064,11.0,nm0000116,director,\N,\N,James Cameron,1954,\N,"writer,producer,director",tt0103064


In [98]:
essai = pd.merge(essai1,
                 df_films,
                 how = 'left',
                 on = 'film_id_out_KNN')

In [100]:
essai['title_out_KNN']

0                                             Avatar
1                           Avatar: La Voie de l'eau
2                                  Aliens, le Retour
3    Exterminador Implacável 2 - O Dia do Julgamento
Name: title_out_KNN, dtype: object

In [84]:
df_final = df_final.drop(['primaryProfession'])

,film_id_out_KNN,tconst,ordering,nconst,category,job,characters,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,tt0040068,tt0040068,12.0,nm0059106,director,director,\N,NaN,NaN,NaN,NaN,NaN
1,tt0869977,tt0869977,11.0,nm1780037,director,\N,\N,Céline Sciamma,1978,\N,"writer,director,actress",tt0869977
2,tt0867594,tt0867594,6.0,nm2919747,director,\N,\N,NaN,NaN,NaN,NaN,NaN
3,tt0867594,tt0867594,7.0,nm2391494,director,\N,\N,Fergus Daly,\N,\N,"director,writer,producer",tt0867594
4,tt1939753,tt1939753,11.0,nm1288736,director,director,\N,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
545429,tt9428746,tt9428746,6.0,nm1204760,actor,\N,"[""Sam Quintana""]",NaN,NaN,NaN,NaN,NaN
545430,tt9428746,tt9428746,7.0,nm3322393,actor,\N,"[""Charlie Escobedo""]",NaN,NaN,NaN,NaN,NaN
545431,tt9428746,tt9428746,8.0,nm2790265,actor,\N,"[""Julian Enriquez""]",NaN,NaN,NaN,NaN,NaN
545432,tt9428746,tt9428746,9.0,nm0287168,actress,\N,"[""Tia Ophelia""]",NaN,NaN,NaN,NaN,NaN


In [32]:
df_title_principals_name = pd.merge(df_title_principals,
                                    df_name_basics_exp,
                                    on = 'nconst',
                                    how = 'left')

: 

: 

In [14]:
df_title_principals = 0

In [15]:
df_title_crew_name = pd.merge(df_title_crew_exp,
                                    df_name_basics_exp,
                                    right_on = 'nconst',
                                    left_on = 'directors',
                                    how = 'left')

In [16]:
df_title_crew_exp = 0

In [17]:
df_title_crew_name = df_title_crew_name.rename(columns = {'primaryName' : 'director_primaryName'})

In [ ]:
df_title_director_name = df_title_crew_name[['directors', 'director_primaryName', 'knownForTitles']]

In [ ]:
df_title_crew_name = 0

In [ ]:
df_films = df_films['film_id_out_KNN']

In [ ]:
df_final = pd.merge(df_films,
                    df_title_director_name,
                    how = 'left',
                    left_on = 'film_id_out_KNN',
                    right_on = 'knownForTitles')

In [19]:
df_final.to_csv('P2_G5_directors.csv.gz', index=False, compression='gzip')

KeyboardInterrupt: 